---
# IMPORT

In [29]:
import os
import numpy as np
import pandas as pd
import ast

from PIL import Image

from tensorflow.keras.models import Model
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dropout, Dense, Input, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

---
# VARIABLES

In [30]:
mapping = {0:'Anger', 1:'Disgust', 2:'Fear', 3:'Happiness', 4:'Neutral', 5:'Sadness', 6:'Surprise'}
name_model = 'EmotionModelLastDayVersion1.h5'
# name_model = 'EmotionModelBest.h5'
bbox_label_file = 'bbox_0.75_publictest.csv'
link_folder = 'public_test'
size_image = (48, 48)

---
# LOAD MODEL

In [31]:
def build_model():    
    inputs = Input(shape=(48, 48, 3))
    base_model = EfficientNetB0(include_top=False, weights='imagenet',
                                drop_connect_rate=0.33, input_tensor=inputs)
    x = GlobalAveragePooling2D()(base_model.output)
    x = Dropout(.5, name="top_dropout")(x)
    outputs = Dense(7, activation='softmax')(x)
    model = Model(inputs, outputs)

    model.compile(optimizer=Adam(learning_rate=1e-3),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [32]:
model = build_model()
model.load_weights(name_model)
model

---
# PREDICT PUBLIC TEST - PRIVATE TEST

In [33]:
def read_file_image(file_path):
    test_data = []
    label_file = pd.read_csv(bbox_label_file)
    for i in range(len(label_file)):
        pixels = Image.open(os.path.join(file_path, label_file.iloc[i]['file_name']))
        bbox = label_file.iloc[i]['bbox']
        bbox = ast.literal_eval(bbox)
        pixels = pixels.crop((bbox[0], bbox[1], bbox[0] + bbox[2], bbox[1] + bbox[3]))
        test_data.append(np.array(pixels.resize(size_image)))  
    return test_data

In [34]:
def preprocess_data_input(data):
    processed_data = []
    for each in data:
        if len(each.shape) == 2:
            each = np.expand_dims(each, axis=-1)
            each = np.concatenate([each, each, each], axis=-1)
        elif each.shape[2] == 1:
            each = np.concatenate([each, each, each], axis=-1)
        processed_data.append(each)
    return np.array(processed_data)

In [35]:
# Read data
data = read_file_image(link_folder)
processed_data = preprocess_data_input(data)

In [36]:
labels = model.predict(processed_data)
labels = np.argmax(np.array(labels), axis=1)

70/70 [==============================] - 7s 67ms/step


In [37]:
bbox_labels = pd.read_csv(bbox_label_file)
result = pd.DataFrame({'file_name': bbox_labels['file_name'], 'bbox': bbox_labels['bbox'], 'emotion': list(map(lambda x: mapping[x], labels))})
result.to_csv('emotion_result(version1).csv', index=False)